# Unknown painting detection threshold optimization

In this notebook the top2/top1 number of paintings ratio is optimized using different keypoint detection systems.

### Imports

In [ ]:
import os
import cv2
import numpy as np
import shutil
import tqdm as tqdm
import pickle
import time
from sklearn.metrics import f1_score

from src.utils.images import load_images_from_directory
from src.utils.denoising import create_denoised_dataset
from src.utils.segmentation import generate_masks
from src.utils.keypoint_descriptors import *

### Helper functions

In [ ]:
def lowe_ratio_test (knn_matches, ratio_threshold):
    """
    Applies Lowe's ratio test to filter out poor matches from k-nearest neighbors (k-NN) match results.

    Args:
        knn_matches (list of tuples): A list of tuples where each tuple contains two matches (m, n).
            - `m` and `n` are typically objects with a `distance` attribute, representing the 
              distance between matched features.
        ratio_threshold (float): The threshold ratio to determine if a match is good. A smaller 
            ratio is more strict and filters out more matches.

    Returns:
        list: A list of good matches that pass Lowe's ratio test. Each match in the list is from 
        the first element of the tuple `m` in `knn_matches` that satisfies the ratio test.
    """
    good_matches = []
    for match_pair in knn_matches:
        if len(match_pair) >= 2:
            m, n = match_pair[0], match_pair[1]
            if m.distance < ratio_threshold * n.distance:
                good_matches.append(m)
    
    return good_matches


def function_time_count(function, params):

    """
    Measures the execution time of a given function and returns both the 
    time taken and the function's result.

    Args:
        function (callable): The function to be executed.
        params (list): A tuple of parameters to pass to the function.

    Returns:
    - tuple: A tuple containing:
        - total_time (float): The time taken to execute the function, in seconds.
        - results: The output of the executed function.
    """

    start = time.time()
    results = function(*params)
    end = time.time()
    total_time = end-start

    return total_time, results


def get_key_des_multi_image(images_list, method):
    """
    Identifies keypoints and calculates descriptors for each image in
    a list of loaded images using the specified method.
    
    Args:
    - images_list (list of ndarray): list of loaded images
    - method (str): method to use to extract the keypoints and descriptors
    
    Returns:
    - key_des_list (list of dictionaries): list of dictionaries, each
                    dictionary containing the keypoints and descriptors
                    for each image.
    """
    
    if method=="SIFT":
        key_des_list = get_SIFT_key_des_multi_image(images_list)
    
    elif method=="ORB":
        key_des_list = get_ORB_key_des_multi_image(images_list)
        
    elif method=="AKAZE":
        key_des_list = get_AKAZE_key_des_multi_image(images_list)
        
    elif method=="Harris-SIFT":
        key_des_list = get_key_des_wildcard_multi_image(images_list, get_Harris_key, get_SIFT_descriptors)
    
    elif method=="Harris-ORB":
        key_des_list = get_key_des_wildcard_multi_image(images_list, get_Harris_key, get_ORB_descriptors)

    elif method=="Harris-AKAZE":
        key_des_list = get_key_des_wildcard_multi_image(images_list, get_Harris_key, get_AKAZE_descriptors)
    
    elif method=="HarrisLaplacian-SIFT":
        key_des_list = get_key_des_wildcard_multi_image(images_list, get_Harris_Laplacian_keypoints, get_SIFT_descriptors)
    
    elif method=="HarrisLaplacian-ORB":
        key_des_list = get_key_des_wildcard_multi_image(images_list, get_Harris_Laplacian_keypoints, get_ORB_descriptors)
    
    elif method=="HarrisLaplacian-AKAZE":
        key_des_list = get_key_des_wildcard_multi_image(images_list, get_Harris_Laplacian_keypoints, get_AKAZE_descriptors)
    
    return key_des_list

def get_num_matching_descriptors(descriptors_image_1, descriptors_image_2, method, descr_method, params=[]):
    """
    Matches descriptors between two images using either Brute-Force or FLANN-based matching.

    Parameters:
        descriptors_image_1: ndarray
            Descriptors from the first image.
        descriptors_image_2: ndarray
            Descriptors from the second image.
        method: str
            Matching method to use. Options:
            - "BruteForce": Uses Brute-Force matcher.
            - "FLANN": Uses FLANN-based matcher.
        descr_method: str
            Descriptor method used for extracting features. Options:
            - "SIFT": Uses floating-point descriptors.
            - "ORB", "AKAZE": Use binary descriptors.
        params: list, optional
            Additional parameters depending on the method:
            - For "BruteForce":
                params[0]: int
                    Norm type (default: cv2.NORM_L2 for SIFT, cv2.NORM_HAMMING for ORB/AKAZE).
                params[1]: bool
                    Whether to use crossCheck (default = False).
            - For "FLANN":
                params[0]: dict
                    Index parameters.
                params[1]: dict
                    Search parameters.
                params[2]: int
                    Number of nearest neighbors (k) (default: 5).
                params[3]: float
                    Lowe's ratio for filtering matches (default: 0.7).

    Returns:
    - tuple:
        - matches: list
            List of matched descriptors.
        - num_matches: int
            The number of matches found.

    Notes:
    - BruteForce:
        - Uses Euclidean distance for SIFT.
        - Uses Hamming distance for ORB and AKAZE.
    - FLANN:
        - Uses KDTree for SIFT.
        - Uses LSH for ORB and AKAZE.
        - Applies Lowe's ratio test for FLANN-based matches.
    """
    if method == "BruteForce":
        if descr_method in ["SIFT", "Harris-SIFT", "HarrisLaplacian-SIFT"]:
            if params:
                norm = params[0]
                crossCheck = params[1]
            else:
                norm = cv2.NORM_L2
                crossCheck = False

        elif descr_method in ["ORB","AKAZE", "Harris-ORB", "Harris-AKAZE", "HarrisLaplacian-ORB", "HarrisLaplacian-AKAZE"]:
            if params:
                norm = params[0]
                crossCheck = params[1]
            else:
                norm = cv2.NORM_HAMMING
                crossCheck = False
        else:
            norm = cv2.NORM_HAMMING
            crossCheck = True

        matcher = cv2.BFMatcher(norm, crossCheck)
        matches = matcher.match(descriptors_image_1, descriptors_image_2)
        num_matches = len(matches)

    elif method == "FLANN":
        if descr_method == "SIFT":
            if params:
                index_params = params[0]
                search_params = params[1]
                k = params[2]
                ratio = params[3]
            else:
                index_params = dict(algorithm=1, trees=5)
                search_params = dict(checks=50)
                k = 2
                ratio = 0.7

        elif descr_method in ["ORB","AKAZE"]:
            if params:
                index_params = params[0]
                search_params = params[1]
                k = params[2]
                ratio = params[3]
            else:
                index_params = dict(algorithm=6, table_number=6, key_size=12, multi_probe_level=1)
                search_params = dict(checks=50)
                k = 2
                ratio = 0.7
        
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
        knn_matches = matcher.knnMatch(descriptors_image_1, descriptors_image_2, k)
        matches = lowe_ratio_test(knn_matches, ratio)
        num_matches = len(matches)

    return matches, num_matches


def check_for_unknown_painting(num_matching_descriptors_list, unknown_painting_threshold):
    """
    Determines whether an unknown painting is present by analyzing the ratio of matching descriptors.

    Args:
        num_matching_descriptors_list (list of int): A list containing the number of matching 
            descriptors for each known painting.
        unknown_painting_threshold (float): The threshold ratio used to determine if the painting 
            is unknown. If the ratio of the second highest to the highest number of matches exceeds 
            this threshold, the painting is considered unknown.

    Returns:
        bool: True if the painting is determined to be unknown, False otherwise.
    """

    matching_list_aux = sorted(num_matching_descriptors_list, reverse=True)
    max_matches = matching_list_aux[0]
    second_max_matches = matching_list_aux[1]

    ratio = second_max_matches / (max_matches+0.000001)

    return ratio


def get_num_matches(query_dir, bbdd_dir, method, matching_method, matching_params=[],
                    cache_segmented=False, cache_denoised=False):
    """
    Processes query images to identify paintings by matching their features against a database (BBDD),
    using specified image processing and matching methods. Only returns the number of matching descriptors
    per query per bbdd in a list of lists.

    This function involves several steps: denoising, segmentation, feature extraction, and descriptor 
    matching. The process is optimized with optional caching for segmentation and denoising.

    Args:
        query_dir (str): Path to the directory containing query images.
        bbdd_dir (str): Path to the directory containing database (BBDD) images.
        method (str): Method used for extracting keypoints and descriptors (e.g., SIFT, ORB).
        matching_method (str): Method used for matching descriptors (e.g., brute-force, FLANN).
        matching_params (list, optional): Additional parameters for the matching method.
        unknown_painting_threshold (float, optional): Threshold for determining if a painting is unknown.
            Default is 2.
        cache_segmented (bool, optional): If True, uses cached segmented images. Default is False.
        cache_denoised (bool, optional): If True, uses cached denoised images. Default is False.

    Returns:
        list: A list of lists with the number of matching descriptors between each query and
              database image.
    """
        
    if cache_segmented:
        print("Using cached segmented images.")
        
    if cache_denoised:
        print("Using cached denoised images.")
    
    # REMOVE NOISE FROM QUERY IMAGES FOR SEGMENTATION
    # ======================================================================================

    # Create a new directory for denoised images
    denoised_for_segmentation_queries_dir = 'data/denoised_for_segmentation_queries_dir'

    if not cache_segmented:
        # Remove previous denoised images
        if os.path.exists(denoised_for_segmentation_queries_dir):
            shutil.rmtree(denoised_for_segmentation_queries_dir)

        # Execute denoising method 1
        create_denoised_dataset(
            noisy_dataset_path = query_dir,
            denoised_dataset_path = denoised_for_segmentation_queries_dir,
            method='gaussian',
            lowpass_params={'ksize': 3},
            highpass=False
        )

        # Read denoised images
        rgb_queries_denoised = []
        for filename in os.listdir(denoised_for_segmentation_queries_dir):
            if filename.endswith('.jpg'):
                img_path = os.path.join(denoised_for_segmentation_queries_dir, filename)
                img_rgb = cv2.imread(img_path)
                if img_rgb is not None:
                    rgb_queries_denoised.append(img_rgb)
                else:
                    print(f"Warning: Failed to read {img_path}")


    # DETECT PAINTINGS IN QUERIES (SEGMENTATION)
    # ======================================================================================

    masks_queries_dir = "data/masks_queries_dir"
    cropped_queries_dir = "data/cropped_queries_dir"

    if not cache_segmented:
        # Remove previous masks and cropped images
        if os.path.exists(masks_queries_dir):
            shutil.rmtree(masks_queries_dir)
        if os.path.exists(cropped_queries_dir):
            shutil.rmtree(cropped_queries_dir)

        # Create new directories for masks and cropped images
        os.makedirs(masks_queries_dir, exist_ok=True)
        os.makedirs(cropped_queries_dir, exist_ok=True)

        masks = generate_masks(rgb_queries_denoised)

        # Read query images
        rgb_queries = []
        for filename in os.listdir(query_dir):
            if filename.endswith('.jpg'):
                img_path = os.path.join(query_dir, filename)
                img_rgb = cv2.imread(img_path)
                if img_rgb is not None:
                    rgb_queries.append(img_rgb)
                else:
                    print(f"Warning: Failed to read {img_path}")

        paintings_per_image = []
        image_counter = 0

        for i, mask in enumerate(tqdm(masks, desc="Generating segmentation masks and cropping images")):
            # Save the mask
            mask_filename = f"{i:05d}.png"
            masks_save_path = os.path.join(masks_queries_dir, mask_filename)
            cv2.imwrite(masks_save_path, mask)

            # Detect connected components in the mask
            num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)

            # Count paintings (objects of interest) in the current image
            painting_count = 0

            # Collect all valid components (ignoring background label 0)
            components = []

            for j in range(1, num_labels):
                x, y, w, h, area = stats[j]

                components.append((x, y, w, h, area))  # Store component details

            # Sort components from leftmost to rightmost (higher x to lower x)
            components_sorted = sorted(components, key=lambda c: c[0])

            # Iterate over sorted components and save them
            for (x, y, w, h, area) in components_sorted:
                # Extract and save the cropped region
                cropped_result = rgb_queries[i][y:y+h, x:x+w]
                cropped_filename = f"{image_counter:05d}.jpg"
                cropped_save_path = os.path.join(cropped_queries_dir, cropped_filename)
                cv2.imwrite(cropped_save_path, cropped_result)

                # Increment the counter for unique naming
                image_counter += 1
                painting_count += 1

            # Add the number of paintings detected for this image to the list
            paintings_per_image.append(painting_count)

        # Save the list of frame counts per image in a single .pkl file
        with open("data/paintings_per_image.pkl", "wb") as f:
            pickle.dump(paintings_per_image, f)


    # REMOVE NOISE FROM QUERY IMAGES
    # =======================================================================================

    # Load list containing paintings per image
    with open('data/paintings_per_image.pkl', 'rb') as file:
        paintings_per_image = pickle.load(file)

    denoised_paintings_folder = 'data/denoised_paintings'

    if not cache_denoised:
        # Remove previous paintings
        if os.path.exists(denoised_paintings_folder):
            shutil.rmtree(denoised_paintings_folder)

        # Create new temporary directory for denoised images
        os.makedirs(denoised_paintings_folder, exist_ok=True)

        # Using denoising method 5
        create_denoised_dataset(
            noisy_dataset_path = cropped_queries_dir,
            denoised_dataset_path = denoised_paintings_folder,
            method='wavelet',
            wavelet_params={'wavelet':'db1', 'mode':'soft', 'rescale_sigma':True},
            highpass=False
        )


    # EXTRACT LOCAL FEATURES (KEYPOINT DESCRIPTORS) FROM
    # DENOISED QUERIES AND BBDD
    # =======================================================================================

    # Load denoised query paintings and bbdd images
    query_images = load_images_from_directory(denoised_paintings_folder)
    bbdd_images = load_images_from_directory(bbdd_dir)

    # Extract keypoints and descriptors
    query_key_des_list = get_key_des_multi_image(query_images, method)
    bbdd_key_des_list = get_key_des_multi_image(bbdd_images, method)
    
    # Results matrix
    full_matching_descriptors = []
    
    # For each query
    for query_image in tqdm(query_key_des_list, desc="Matching descriptors"):
        
        # Get matching descriptors from each bbdd image
        num_matching_descriptors_list = []
        for bbdd_image in bbdd_key_des_list:
            
            # There must be at least one descriptor in the bbdd image
            if str(bbdd_image['descriptors'])!="None":
                _, num_matching_descriptors = get_num_matching_descriptors(query_image['descriptors'],
                                                                           bbdd_image['descriptors'],
                                                                           method=matching_method,
                                                                           descr_method=method,
                                                                           params=matching_params)
            else:
                num_matching_descriptors = 0
                
            num_matching_descriptors_list.append(num_matching_descriptors)
        
        full_matching_descriptors.append(num_matching_descriptors_list)
    
    return full_matching_descriptors

def generate_submission(results, paintings_per_image):
    """
    Gets the top 1 bbdd image for each query, taking into account
    whether there were one or two paintings in the image.
    
    Args:
    - results: list of lists with the ordered top results for each query
    - paintings_per_image: list with number of paintings in every query image
    
    Returns:
    - submission: a list of lists in the sumbission format
    """
    
    submission = []
    i = 0
    for num_paintings in paintings_per_image:
        if num_paintings == 1:
            submission.append([results[i][0]])
            i += 1
        elif num_paintings == 2:
            submission.append([results[i][0], results[i+1][0]])
            i += 2
            
    return submission

def order_results_by_num_paintings(results, paintings_per_image):
    
    ordered_results = []
    i = 0
    for num_paintings in paintings_per_image:
        if num_paintings == 1:
            ordered_results.append([results[i]])
            i += 1
        elif num_paintings == 2:
            ordered_results.append([results[i], results[i+1]])
            i += 2
            
    return ordered_results
    
def get_unknowns_f1(submission, ground_truth):
    """
    Calculates the f1 score considering only the predictions regarding
    the unknown paintings.
    
    Args:
    - submission: results in the submission format.
    - ground_truth: groundtruth in the W4 format.
    
    Returns:
    - f1: f1 score regarding the unknown paintings predictions
    """
    
    submission_binary = [1 if sublist[0] == -1 else 0 for sublist in submission]
    groundtruth_binary = [1 if sublist[0] == -1 else 0 for sublist in ground_truth]

    f1 = f1_score(groundtruth_binary, submission_binary)
    
    return f1

def get_top_two_max(full_matching_descriptors):
    top_two_list = []
    for matching_list in full_matching_descriptors:
        top_two = sorted(matching_list, reverse=True)[0:2]
        top_two_list.append(top_two)
    return top_two_list

def remove_multiple_painting_entries(list1, list2):
    
    # Check if both lists are of the same size
    if len(list1) != len(list2):
        raise ValueError("Both lists must have the same size.")
    
    # Iterate over list2 with index to find where elements are sublists of length 2
    indices_to_remove = [i for i, item in enumerate(list2) if isinstance(item, list) and len(item) == 2]
    
    # Remove elements in reverse order to avoid index shifting
    for index in sorted(indices_to_remove, reverse=True):
        del list1[index]
        del list2[index]
    
    return list1, list2

def check_for_unknown_painting_modified(num_matching_descriptors_list, unknown_painting_threshold):
    
    max_matches = num_matching_descriptors_list[0]
    second_max_matches = num_matching_descriptors_list[1]
    
    ratio = second_max_matches / (max_matches + 0.0000001)

    return int(ratio > unknown_painting_threshold)

def get_unknown_vector(predictions_list):
    return [1 if sublist[0] == -1 else 0 for sublist in predictions_list]

def get_unknown_paintings(filtered_matchings, unknown_painting_threshold):
    unknown_predictions = []
    for query in filtered_matchings:
        uknown = check_for_unknown_painting_modified(query[0], unknown_painting_threshold)
        unknown_predictions.append(uknown)
    return unknown_predictions

### Main function that performs the optimization

In [24]:
def execute_unknown_painting_threshold_optimization(query_dir, bbdd_dir,
                                                    method,
                                                    matching_method,
                                                    matching_params=[],
                                                    cache_segmented=False,
                                                    cache_denoised=False):
    
    full_matching_descriptors = get_num_matches(query_dir, bbdd_dir,
                                                method,
                                                matching_method,
                                                matching_params,
                                                cache_segmented,
                                                cache_denoised)
    
    with open("data/paintings_per_image.pkl", "rb") as f:
        paintings_per_image = pickle.load(f)

    with open(os.path.join(query_dir, 'gt_corresps.pkl'), 'rb') as f:
        ground_truth = pickle.load(f)
        
    print("Executing unknown painting threshold optimization for parameters", method, matching_method, matching_params)

    # Preprocess results
    # ===================================================================================================
    # Get the top two number of matchings per query
    top_two_matching_list = get_top_two_max(full_matching_descriptors)

    # Pair paintings corresponding to the same image in a submission-like manner
    ordered_matchings = order_results_by_num_paintings(top_two_matching_list, paintings_per_image)

    # Only consider images with 1 painting, so cropping issues are not taken
    # into account by the threshold optimization process.
    filtered_matchings, filtered_gt = remove_multiple_painting_entries(ordered_matchings, ground_truth)

    # Convert filtered groundtruth to 0s and 1s vectors (1 if unknown, 0 otherwise), so we can
    # later caluclate the f1 more easily
    gt_vector = get_unknown_vector(filtered_gt)
    
    # Optimize threshold
    # ===================================================================================================
    # Initialize an empty list to store the results
    results = []

    # Loop over the threshold grid, from 0.01 to 0.99 with a step of 0.0001
    for threshold in np.arange(0.01, 1.00, 0.0001):
        # Get predictions for the current threshold
        unknown_preds = get_unknown_paintings(filtered_matchings, unknown_painting_threshold=threshold)

        # Calculate F1 score for the current threshold
        f1 = f1_score(gt_vector, unknown_preds)

        # Append the threshold and corresponding F1 score to the results list
        results.append({'threshold': threshold, 'f1_score': f1})

    # Convert results to a DataFrame for easier retrieval and analysis
    results_df = pd.DataFrame(results)

    # Find the threshold with the best (highest) F1 score
    best_result = results_df.loc[results_df['f1_score'].idxmax()]

    # Print the best threshold and corresponding F1 score
    print("Best Threshold:", best_result['threshold'])
    print("Best F1 Score:", best_result['f1_score'])
    
    return top_two_matching_list

### Parameter optimization for different keypoint descriptor methods

In [5]:
query_dir = "./data/qsd1_w4/"
bbdd_dir = "./data/BBDD/"

execute_unknown_painting_threshold_optimization(query_dir, bbdd_dir,
                                                method="ORB",
                                                matching_method="FLANN",
                                                matching_params=[],
                                                cache_segmented=False,
                                                cache_denoised=False)

Denoising images: 100%|████████████████████████████████████████████████████████████████| 94/94 [00:02<00:00, 40.80it/s]
Generating segmentation masks and cropping images: 100%|███████████████████████████████| 30/30 [00:01<00:00, 18.35it/s]
Matching descriptors: 100%|████████████████████████████████████████████████████████████| 37/37 [00:26<00:00,  1.42it/s]


Executing unknown painting threshold optimization for parameters ORB FLANN []
Best Threshold: 0.506199999999997
Best F1 Score: 0.8695652173913044


In [106]:
query_dir = "./data/qsd1_w4/"
bbdd_dir = "./data/BBDD/"

execute_unknown_painting_threshold_optimization(query_dir, bbdd_dir,
                                                method="ORB",
                                                matching_method="BruteForce",
                                                matching_params=[cv2.NORM_HAMMING, True],
                                                cache_segmented=False,
                                                cache_denoised=False)

Denoising images: 100%|████████████████████████████████████████████████████████████████| 94/94 [00:01<00:00, 48.11it/s]
Generating segmentation masks and cropping images: 100%|███████████████████████████████| 30/30 [00:01<00:00, 21.14it/s]
Matching descriptors: 100%|████████████████████████████████████████████████████████████| 37/37 [00:28<00:00,  1.32it/s]


Executing unknown painting threshold optimization for parameters ORB BruteForce [6, True]
Best Threshold: 0.9280999999999945
Best F1 Score: 0.8


In [111]:
query_dir = "./data/qsd1_w4/"
bbdd_dir = "./data/BBDD/"

execute_unknown_painting_threshold_optimization(query_dir, bbdd_dir,
                                                method="AKAZE",
                                                matching_method="BruteForce",
                                                matching_params=[cv2.NORM_HAMMING, True],
                                                cache_segmented=True,
                                                cache_denoised=True)

Using cached segmented images.
Using cached denoised images.


Matching descriptors: 100%|█████████████████████████████████████████████████████████| 37/37 [1:12:02<00:00, 116.82s/it]


Executing unknown painting threshold optimization for parameters AKAZE BruteForce [6, True]
Best Threshold: 0.8685999999999948
Best F1 Score: 0.6896551724137931


In [13]:
query_dir = "./data/qsd1_w4/"
bbdd_dir = "./data/BBDD/"

execute_unknown_painting_threshold_optimization(query_dir, bbdd_dir,
                                                method="HarrisLaplacian-ORB",
                                                matching_method="BruteForce",
                                                matching_params=[cv2.NORM_HAMMING, True],
                                                cache_segmented=True,
                                                cache_denoised=True)

Using cached segmented images.
Using cached denoised images.


Matching descriptors: 100%|████████████████████████████████████████████████████████████| 37/37 [03:56<00:00,  6.40s/it]


Executing unknown painting threshold optimization for parameters HarrisLaplacian-ORB BruteForce [6, True]
Best Threshold: 0.834499999999995
Best F1 Score: 0.7142857142857143
